# SQL 2
## April 5th, 2022
### Overview: A further excursion into SQL language, now with joining, grouping, etc.

In [1]:
import sqlite3 as sql

In [7]:
# Problem 1
def prob1(db_file="students.db"):
    """Query the database for the list of the names of students who have a
    'B' grade in any course. Return the list.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): a list of strings, each of which is a student name.
    """
    with sql.connect(db_file) as conn:
        cur = conn.cursor()
        
        #getting the names of students who have a B in any course
        Bs = cur.execute("SELECT SI.StudentName "
                   "FROM StudentInfo AS SI INNER JOIN StudentGrades as SG "
                   "ON SI.StudentID=SG.StudentID "
                   "WHERE SG.Grade='B';").fetchall()
    conn.close()
    
    #extracting student name strings from tuples
    B_students = [B[0] for B in Bs]
    
    return B_students

In [10]:
# Problem 2
def prob2(db_file="students.db"):
    """Query the database for all tuples of the form (Name, MajorName, Grade)
    where 'Name' is a student's name and 'Grade' is their grade in Calculus.
    Only include results for students that are actually taking Calculus, but
    be careful not to exclude students who haven't declared a major.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """
    with sql.connect(db_file) as conn:
        cur = conn.cursor()
        
        #left joining student info and major info on major ids, then inner joining with student grades on student ids
        #getting student names, their major, and their grades
        students = cur.execute("SELECT SI.StudentName, MI.MajorName, SG.Grade "
                    "FROM StudentInfo AS SI LEFT OUTER JOIN MajorInfo AS MI "
                    "ON SI.MajorID=MI.MajorID "
                    "INNER JOIN StudentGrades AS SG "
                    "ON SI.StudentID=SG.StudentID "
                    "WHERE SG.CourseID=1;").fetchall()
    conn.close()
        
    return students

In [57]:
# Problem 3
def prob3(db_file="students.db"):
    """Query the database for the list of the names of courses that have at
    least 5 students enrolled in them.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): a list of strings, each of which is a course name.
    """
    with sql.connect(db_file) as conn:
        cur = conn.cursor()
        
        #getting courses that have at least five students
        fives = cur.execute("SELECT CI.CourseName "
                    "FROM CourseInfo as CI INNER JOIN StudentGrades AS SG "
                    "ON CI.CourseID=SG.CourseID "
                    "GROUP BY SG.CourseID "
                    "HAVING COUNT(*)>=5;").fetchall()
        
    conn.close()
    
    #getting just the course names from the tuples
    Fives = [f[0] for f in fives]
    return Fives

In [3]:
# Problem 4
def prob4(db_file="students.db"):
    """Query the given database for tuples of the form (MajorName, N) where N
    is the number of students in the specified major. Sort the results in
    descending order by the counts N, then in alphabetic order by MajorName.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """
    with sql.connect(db_file) as conn:
        cur = conn.cursor()
        
        #getting major names and the number of students majoring in them, then ordering them
        results = cur.execute("SELECT MI.MajorName, COUNT(*) AS num_studs "
                              "FROM StudentInfo as SI LEFT OUTER JOIN MajorInfo as MI "
                              "ON MI.MajorID=SI.MajorID "
                              "GROUP BY SI.MajorID "
                              "ORDER BY num_studs DESC, MI.MajorName ASC;").fetchall()
    conn.close()
    
    return results

In [4]:
prob4()

[(None, 3), ('Science', 3), ('Math', 2), ('Art', 1), ('Writing', 1)]

In [75]:
# Problem 5
def prob5(db_file="students.db"):
    """Query the database for tuples of the form (StudentName, MajorName) where
    the last name of the specified student begins with the letter C.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """
    with sql.connect(db_file) as conn:
        cur = conn.cursor()
        
        #getting students and their majors, where the students' last names start with C
        results = cur.execute("SELECT SI.StudentName, MI.MajorName "
                              "FROM StudentInfo as SI OUTER LEFT JOIN MajorInfo as MI "
                              "ON SI.MajorID=MI.MajorID "
                              "WHERE SI.StudentName LIKE '% C%';").fetchall()
    conn.close()
    
    return results

In [109]:
# Problem 6
def prob6(db_file="students.db"):
    """Query the database for tuples of the form (StudentName, N, GPA) where N
    is the number of courses that the specified student is in and 'GPA' is the
    grade point average of the specified student according to the following
    point system.

        A+, A  = 4.0    B  = 3.0    C  = 2.0    D  = 1.0
            A- = 3.7    B- = 2.7    C- = 1.7    D- = 0.7
            B+ = 3.4    C+ = 2.4    D+ = 1.4

    Order the results from greatest GPA to least.

    Parameters:
        db_file (str): the name of the database to connect to.

    Returns:
        (list): the complete result set for the query.
    """
    with sql.connect(db_file) as conn:
        cur = conn.cursor()
        
        
        #note to me: we're making this new table with the cases, and we're doing so by using the SG table, but then we 
        #also give this cased table the alias SG
        results = cur.execute("SELECT SI.StudentName, COUNT(*), AVG(SG.points) "
                              "FROM ("
                                  "SELECT StudentID, CASE Grade "
                                      "WHEN 'A+' THEN 4.0 "
                                      "WHEN 'A' THEN 4.0 "
                                      "WHEN 'A-' THEN 3.7 "
                                      "WHEN 'B+' THEN 3.4 "
                                      "WHEN 'B' THEN 3.0 "
                                      "WHEN 'B-' THEN 2.7 "
                                      "WHEN 'C+' THEN 2.4 "
                                      "WHEN 'C' THEN 2.0 "
                                      "WHEN 'C-' THEN 1.7 "
                                      "WHEN 'D+' THEN 1.4 "
                                      "WHEN 'D' THEN 1.0 "
                                      "WHEN 'D-' THEN 0.7 END AS points "
                                 "FROM StudentGrades) AS SG "
                              "INNER JOIN StudentInfo as SI "
                              "ON SI.StudentID=SG.StudentID "
                              "GROUP BY SG.StudentID;").fetchall()
        
    
    conn.close()
    
    return results

In [110]:
prob6()

[('Cameron Kim', 3, 2.566666666666667),
 ('Kristopher Tran', 2, 3.2),
 ('Alfonso Phelps', 2, 2.5),
 ('Cassandra Holland', 2, 2.2),
 ('Michelle Fernandez', 2, 2.35),
 ('Roberta Cook', 1, 2.0),
 ('Sammy Burke', 2, 3.85),
 ('Gilbert Chapman', 3, 3.6666666666666665),
 ('Mercedes Hall', 3, 2.6666666666666665),
 ('Rene Cross', 1, 4.0)]

In [ ]:
results = cur.execute("SELECT SI.StudentName, CASE SG.Grade "
                                  "WHEN 'A+' THEN 4.0 "
                                  "WHEN 'A' THEN 4.0 "
                                  "WHEN 'A-' THEN 3.7 "
                                  "WHEN 'B+' THEN 3.4 "
                                  "WHEN 'B' THEN 3.0 "
                                  "WHEN 'B-' THEN 2.7 "
                                  "WHEN 'C+' THEN 2.4 "
                                  "WHEN 'C' THEN 2.0 "
                                  "WHEN 'C-' THEN 1.7 "
                                  "WHEN 'D+' THEN 1.4 "
                                  "WHEN 'D' THEN 1.0 "
                                  "WHEN 'D-' THEN 0.7 END "
                             "FROM StudentGrades as SG INNER JOIN StudentInfo as SI "
                             "ON SI.StudentID=SG.StudentID;").fetchall()